# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 4

## О документе

Версия 1.03

Модуль 4 для регионального этапа соревнований по кибериммунной автономности

### Модуль 4. Следование по трассе с киберпрепятствиями

Для успешного выполнения этого задания необходимое активировать специальный режим работы виртуальной машинки - в рамках этого задания можно менять только блоки, отвечающие за безопасность (ограничитель, монитор безопасности), **другие блоки менять запрещено**.

При прохождении маршрута будут имитироваться атаки со стороны злоумышленников, которые будут пытаться нарушить цели безопасности. Важно, чтобы им это не удалось.

1. Запустите свою машинку и убедитесь, что она проходит всю трассу без нарушений ограничений скорости. При необходимости измените логику работы блока безопасности
2. Добавьте контроль доставки груза в модуле SafetyBlock - убедитесь, что груз доставляется до конечной точки маршрута.
3. Если в модуле 3 вы реализовали монитор безопасности - не забудьте его перенести в этот модуль, это принесёт дополнительные баллы!

Активация киберпрепятствий в системе управления:

после инициализации системы управления добавьте следующую строку
```python
control_system.enable_surprises()
```

В этом блоке добавьте все ваши реализации изменённых бортовых систем

In [1]:
from src.config import (
    CONTROL_SYSTEM_QUEUE_NAME, 
    SAFETY_BLOCK_QUEUE_NAME,
    SERVOS_QUEUE_NAME,
    CARGO_BAY_QUEUE_NAME,
    LOG_ERROR, 
    LOG_INFO,
    SECURITY_MONITOR_QUEUE_NAME
)
from src.event_types import Event
from src.queues_dir import QueuesDirectory
from queue import Queue
from src.control_system import BaseControlSystem
from src.navigation_system import BaseNavigationSystem
from src.communication_gateway import BaseCommunicationGateway
from src.safety_block import BaseSafetyBlock
from src.security_monitory import BaseSecurityMonitor
from src.security_policy_type import SecurityPolicy
from src.config import (
    COMMUNICATION_GATEWAY_QUEUE_NAME, 
    CONTROL_SYSTEM_QUEUE_NAME, 
    NAVIGATION_QUEUE_NAME,
    SAFETY_BLOCK_QUEUE_NAME,
    SERVOS_QUEUE_NAME,
    CARGO_BAY_QUEUE_NAME,
    LOG_DEBUG, 
    LOG_ERROR, 
    LOG_INFO
)
from src.event_types import Event
from src.queues_dir import QueuesDirectory
from queue import Queue
import math
from src.queues_dir import QueuesDirectory
from src.security_policy_type import SecurityPolicy

from src.config import COMMUNICATION_GATEWAY_QUEUE_NAME, LOG_DEBUG, LOG_ERROR, LOG_INFO

# ============================================================================
# ОГРАНИЧИТЕЛЬ (SAFETY BLOCK)
# ============================================================================

class SafetyBlock(BaseSafetyBlock):
    """ 
    Класс ограничителя безопасности
    
    Обеспечивает:
    - ЦБ1: соблюдение ограничений скорости и направления
    - ЦБ3: контроль доставки груза только в конечной точке маршрута
    """
    
    def _set_new_direction(self, direction: float):
        """ 
        Установка нового направления перемещения с проверкой безопасности
        
        Проверяет, что запрошенное направление соответствует расчетному
        направлению к следующей точке маршрута
        """
        self._log_message(LOG_INFO, f"текущие координаты: {self._position}")
        self._log_message(LOG_DEBUG, f"маршрутное задание: {self._mission}")
        self._log_message(LOG_DEBUG, f"состояние маршрута: {self._route}")
        
        # Если маршрут завершен, устанавливаем направление 0
        if self._route.route_finished:
            self._direction = 0
            self._send_direction_to_consumers()
            return
            
        # Рассчитываем правильное направление к следующей точке
        next_point = self._route.next_point()
        correct_direction = self._calculate_bearing(self._position, next_point)
        
        # Проверяем, что запрошенное направление близко к правильному
        # Допускаем отклонение до 5 градусов
        direction_diff = abs(direction - correct_direction)
        if direction_diff > 5 and direction_diff < 355:
            self._log_message(LOG_ERROR, 
                f"принудительная установка направления! Запрошенное направление {direction}, "
                f"расчётное {correct_direction}")
            # Устанавливаем правильное направление
            self._direction = correct_direction
        else:
            # Направление допустимо
            self._direction = direction
            
        self._send_direction_to_consumers()
    
    def _set_new_speed(self, speed: float):
        """ 
        Установка новой скорости с проверкой безопасности
        
        Проверяет, что запрошенная скорость не превышает ограничения
        для текущего сегмента маршрута
        """
        # Если маршрут завершен, устанавливаем скорость 0
        if self._route.route_finished:
            self._speed = 0
            self._send_speed_to_consumers()
            return
            
        # Получаем ограничение скорости для текущего сегмента
        speed_limit = self._route.calculate_speed()
        
        # Проверяем, что запрошенная скорость не превышает ограничение
        if speed > speed_limit:
            self._log_message(LOG_ERROR, 
                f"принудительная установка скорости! Запрошенная скорость {speed}, "
                f"ограничение {speed_limit}")
            # Устанавливаем допустимую скорость
            self._speed = speed_limit
        else:
            # Скорость допустима
            self._speed = speed
            
        self._send_speed_to_consumers()
    
    def _lock_cargo(self, _):
        """ 
        Блокировка грузового отсека
        
        Разрешаем блокировку в любой момент
        """
        self._send_lock_cargo_to_consumers()
    
    def _release_cargo(self, _):
        """ 
        Разблокировка грузового отсека
        
        Разрешаем только в конечной точке маршрута
        """
        if self._route.route_finished:
            self._log_message(LOG_INFO, "маршрут завершен, разрешаем выгрузку груза")
            self._send_release_cargo_to_consumers()
        else:
            self._log_message(LOG_ERROR, "попытка выгрузки груза вне конечной точки маршрута!")
    
    def _send_speed_to_consumers(self):
        """ Отправка команды скорости в сервоприводы """
        self._log_message(LOG_DEBUG, "отправляем скорость получателям")
        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # Отправка сообщения с допустимой скоростью
        event_speed = Event(
            source=self.event_source_name,
            destination=servos_q_name,
            operation="set_speed",
            parameters=self._speed
        )
        servos_q.put(event_speed)

    def _send_direction_to_consumers(self):
        """ Отправка команды направления в сервоприводы """
        self._log_message(LOG_DEBUG, "отправляем направление получателям")
        servos_q_name = SERVOS_QUEUE_NAME
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # Отправка сообщения с допустимым направлением
        event_direction = Event(
            source=self.event_source_name,
            destination=servos_q_name,
            operation="set_direction",
            parameters=self._direction
        )
        servos_q.put(event_direction)
        
    def _send_lock_cargo_to_consumers(self):
        """ Отправка команды блокировки груза """
        self._log_message(LOG_DEBUG, "отправляем команду блокировки груза")
        cargo_q_name = CARGO_BAY_QUEUE_NAME
        cargo_q: Queue = self._queues_dir.get_queue(cargo_q_name)

        # Отправка сообщения о блокировке груза
        event_lock = Event(
            source=self.event_source_name,
            destination=cargo_q_name,
            operation="lock_cargo",
            parameters=None
        )
        cargo_q.put(event_lock)
        
    def _send_release_cargo_to_consumers(self):
        """ Отправка команды разблокировки груза """
        self._log_message(LOG_DEBUG, "отправляем команду разблокировки груза")
        cargo_q_name = CARGO_BAY_QUEUE_NAME
        cargo_q: Queue = self._queues_dir.get_queue(cargo_q_name)

        # Отправка сообщения о разблокировке груза
        event_release = Event(
            source=self.event_source_name,
            destination=cargo_q_name,
            operation="release_cargo",
            parameters=None
        )
        cargo_q.put(event_release)
        
    def _calculate_bearing(self, start, end):
        """ 
        Расчет направления (азимута) от начальной точки к конечной
        
        Args:
            start: начальная точка (GeoPoint)
            end: конечная точка (GeoPoint)
            
        Returns:
            float: направление в градусах (0-360)
        """
        delta_longitude = end.longitude - start.longitude
        x = math.sin(math.radians(delta_longitude)) * math.cos(math.radians(end.latitude))
        y = math.cos(math.radians(start.latitude)) * math.sin(math.radians(end.latitude)) - \
            math.sin(math.radians(start.latitude)) * math.cos(math.radians(end.latitude)) * \
            math.cos(math.radians(delta_longitude))
        
        initial_bearing_rad = math.atan2(x, y)
        initial_bearing_deg = math.degrees(initial_bearing_rad)
        compass_bearing = (initial_bearing_deg + 360) % 360
        
        return compass_bearing


# ============================================================================
# КОММУНИКАЦИОННЫЙ ШЛЮЗ (COMMUNICATION GATEWAY)
# ============================================================================

class CommunicationGateway(BaseCommunicationGateway):
    """ 
    Класс коммуникационного шлюза
    
    Отправляет маршрутное задание в систему управления через монитор безопасности
    """
    
    def _send_mission_to_consumers(self):
        """ 
        Отправка маршрутного задания получателям через монитор безопасности
        """
        # Создаем событие для системы управления
        self._log_message(1, 'Отправлен Евент')
        event_safety = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="set_mission",
            parameters=self._mission
        )

        event_control = Event(
            source=self.event_source_name,
            destination=CONTROL_SYSTEM_QUEUE_NAME,
            operation="set_mission",
            parameters=self._mission
        )
        
        safety_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        safety_q.put(event_safety)
        control_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        control_q.put(event_control)
        
        #security_monitor_q: Queue = self._queues_dir.get_queue(SAFETY_BLOCK_QUEUE_NAME)
        #security_monitor_q.put(event)


# ============================================================================
# СИСТЕМА НАВИГАЦИИ (NAVIGATION SYSTEM)
# ============================================================================

class NavigationSystem(BaseNavigationSystem):
    """ 
    Класс навигационного блока
    
    Отправляет координаты в систему управления через монитор безопасности
    """
    
    def _send_position_to_consumers(self):
        """ 
        Отправка координат получателям через монитор безопасности
        """
        # Создаем событие для системы управления
        event_control = Event(
            source=self.event_source_name,
            destination=CONTROL_SYSTEM_QUEUE_NAME,
            operation="position_update",
            parameters=self._position
        )
        
        event_safety = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="position_update",
            parameters=self._position
        )

        # Отправляем событие через монитор безопасности
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_q.put(event_safety)
        security_q.put(event_control)

# ============================================================================
# СИСТЕМА УПРАВЛЕНИЯ (CONTROL SYSTEM)
# ============================================================================

class ControlSystem(BaseControlSystem):
    """ 
    Класс системы управления
    
    Отправляет команды скорости и направления в ограничитель через монитор безопасности
    """
    
    def _send_speed_and_direction_to_consumers(self, speed: float, direction: float):
        """ 
        Отправка команд скорости и направления в ограничитель через монитор безопасности
        """
        # Создаем события для ограничителя
        speed_event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="set_speed",
            parameters=speed
        )
        
        direction_event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="set_direction",
            parameters=direction
        )
        
        # Отправляем события через монитор безопасности
        security_monitor_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(speed_event)
        security_monitor_q.put(direction_event)
    
    def _release_cargo(self):
        """ 
        Отправка команды разблокировки груза в ограничитель через монитор безопасности
        """
        # Создаем событие для ограничителя
        event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="release_cargo",
            parameters=None
        )
        
        # Отправляем событие через монитор безопасности
        security_monitor_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)
    
    def _lock_cargo(self):
        """ 
        Отправка команды блокировки груза в ограничитель через монитор безопасности
        """
        # Создаем событие для ограничителя
        event = Event(
            source=self.event_source_name,
            destination=SAFETY_BLOCK_QUEUE_NAME,
            operation="lock_cargo",
            parameters=None
        )
        security_monitor_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        security_monitor_q.put(event)
    

        
class SecurityMonitor(BaseSecurityMonitor):
   
    def __init__(self, queues_dir: QueuesDirectory):
        """
        Иницилиация полити безопаснсоти по умолчанию

        Переопрделение типа переменной self__security_policies с словаря на список
        """
        super().__init__(queues_dir)
        self._security_policies = set()
        self._init_set_security_policies()
     
    
    def _init_set_security_policies(self):
        """ 
        Инициализация политик безопасности
        
        Определяет разрешенные взаимодействия между компонентами системы
        """
        # Базовые политики безопасности
        default_policies = [
            # Коммуникационный шлюз -> Система управления: установка маршрутного задания
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='set_mission'),

            # Коммуникационный шлюз -> Огрначительный блок: установка маршрутного задания
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_mission'), 

            # Система управления -> Ограничитель: установка скорости
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_speed'),
                
            # Система управления -> Ограничитель: установка направления
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_direction'),

            # Система Навигации -> Ограничетльный блок: установка направления
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='position_update'),

            # Система навигации -> Блок Управления: установка направления
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='position_update'),

            # Система управления -> Ограничитель: блокировка груза
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='lock_cargo'),
                
            # Система управления -> Ограничитель: разблокировка груза
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='release_cargo'),
                
            # Ограничитель -> Сервоприводы: установка скорости
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_speed'),
                
            # Ограничитель -> Сервоприводы: установка направления
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_direction'),
                
            # Ограничитель -> Грузовой отсек: блокировка груза
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='lock_cargo'),
                
            # Ограничитель -> Грузовой отсек: разблокировка груза
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='release_cargo'),
        ]
        
        self.set_security_policies(policies=default_policies)
    
    def set_security_policies(self, policies):
        """ 
        Установка новых политик безопасности
        
        Args:
            policies: список политик безопасности
        """
        self._security_policies = policies
        self._log_message(
            LOG_INFO, f"изменение политик безопасности: {policies}")
    
    def _check_event(self, event: Event):
        """ 
        Проверка входящих событий на соответствие политикам безопасности
        
        Args:
            event: проверяемое событие
            
        Returns:
            bool: True если событие разрешено, False если запрещено
        """
        self._log_message(
            LOG_DEBUG, f"проверка события {event}, по умолчанию выполнение запрещено")
        
        authorized = False
     
        # Проверяем, есть ли соответствующая политика
        for policy in self._security_policies:
            if (policy.source == event.source and 
                policy.destination == event.destination and 
                policy.operation == event.operation):
                self._log_message(
                    LOG_DEBUG, "событие разрешено политиками, выполняем")
                authorized = True
                break
                
            self._log_message(
                LOG_DEBUG, "событие разрешено политиками, выполняем")
        
        if authorized is False:
            self._log_message(LOG_ERROR, f"событие не разрешено политиками безопасности! {event}")
            
        return authorized 
    

Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [2]:
afcs_present = True

Поменяем идентификатор машинки для этого модуля

In [3]:
car_id = "m4"

В следующем блоке измените маршрут на ваш, его можно скопировать из модуля 2.
 
Для проверки работы систем безопасности будут активированы киберпрепятствия

```python
control_system.enable_surprises()
```

In [6]:
# используем то же маршрутное задание, которое было в модуле 2
from time import sleep

from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.mission_planner import Mission
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer
from src.wpl_parser import WPLParser
from src.mission_type import GeoSpecificSpeedLimit


# для примера возьмём маршрут из модуля 2
# следует заменить на маршрут, требуемый по регламенту!
wpl_file = "module2.wpl"

parser = WPLParser(wpl_file)    
points = parser.parse()

# обновите скоростные ограничения для вашего маршрута!
speed_limits = [
    GeoSpecificSpeedLimit(0, 30),
    GeoSpecificSpeedLimit(1, 60),
    GeoSpecificSpeedLimit(5, 20),
]


home = points[0]
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 


# создание блоков передачи данных в СУПА
if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

# создание основных функциональных блоков
mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)

communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

safety_block = SafetyBlock(queues_dir=queues_dir, log_level=LOG_INFO)

security_monitor = SecurityMonitor(queues_dir)

# сборка всех запускаемых блоков в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security_monitor
    ] if afcs_present else [
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security_monitor
    ])

#################################
# АКТИВАЦИЯ КИБЕРПРЕПЯТСТВИЙ
control_system.enable_surprises()
#################################

# запуск всех блоков
system_components.start()

# ограничение поездки по времени
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(110)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][QUEUES] регистрируем очередь safety
[ИНФО][SAFETY] создан ограничитель
[ИНФО][QUEUES] регистрируем очередь security
[ИНФО][SECURITY] создан монитор безопасности
[ИНФО][SECURITY] изменение политик безопасности: [SecurityPolicy(source='communication', destination='control', operation='set_mission'), SecurityPolicy(source='communication', destination='safety', operation='set_m

Убедитесь, что 
1. ваша машинка успешно прошла весь заданный маршрут
2. не превысила ограничения скорости
3. успешно доставила груз

Если всё так - поздравляем, вы справились с заданием! Обязательно зафиксируйте все изменения в репозитории!



Мы будем признательны за обратную связь - любые комментарии, которые вы можете дать по итогам выполнения этого задания. 

Например, 

- было ли задание понятным по шкале 1..10 (1 - ничего не понятно, 10 - вопросов вообще не было, всё понятно)?
- было ли задание интересным по шкале 1..10 (1 - скука смертная, 10 - лучшее, что вам пока встречалось на олимпиадах)? 
- что бы вы предложили изменить, чтобы сделает его более интересным?
- по шкале 1..10 насколько сложным оно было для вас?
- что было самым трудным в задании? 

Авторы наиболее развёрнутых и интересных комментариев получат особенный приз от Лаборатории Касперского!

Дополнительная информация о кибериммунной разработке
- https://os.kaspersky.ru/cyber-immune-development/ 
- https://github.com/sergey-sobolev/cyberimmune-systems/wiki/%D0%9A%D0%B8%D0%B1%D0%B5%D1%80%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%B8%D1%82%D0%B5%D1%82
- канал в телеграм: https://t.me/learning_cyberimmunity
